In [43]:
import Control.Applicative
import Data.Char

In [2]:
{-# LANGUAGE InstanceSigs #-}

<h4>Identity Transformer</h4>

In [3]:
newtype Identity a = Identity { runIdentity :: a } deriving (Eq, Show)

-- Identity monad transformer, specifying additional structure should exist
newtype IdentityT f a = IdentityT { runIdentityT :: f a } deriving (Eq, Show)

In [4]:
instance Functor Identity where
    fmap f (Identity a) = Identity $ f a
    
instance Functor m => Functor (IdentityT m) where
    fmap :: (a -> b) -> IdentityT m a -> IdentityT m b
    fmap f (IdentityT ma) = IdentityT $ fmap f ma

In [5]:
instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity $ f a
    
instance Applicative m => Applicative (IdentityT m) where
    pure = IdentityT . pure
    
    (<*>) :: IdentityT m (a -> b) -> IdentityT m a -> IdentityT m b
    (<*>) (IdentityT mf) (IdentityT ma) = IdentityT $ mf <*> ma

In [6]:
instance Monad Identity where
    return = pure
    
    (>>=) :: Identity a -> (a -> Identity b) -> Identity b
    (>>=) (Identity a) f = f a
    
instance Monad m => Monad (IdentityT m) where
    return = IdentityT . return
    
    (>>=) :: IdentityT m a -> (a -> IdentityT m b) -> IdentityT m b
    (>>=) (IdentityT ma) f = IdentityT $ ma >>= runIdentityT . f
    
{--
The key point is the usage of runIdentityT, which enabled us to extract (f a) from the structure.
--}

<div>
General pattern is we have two polymorphic types, f and g, that each have a Monad instance. But, we end up with this pattern:
    
    <code>
        f (g (f b))
    </code><br/>
   
Monads cannot join these types, because of the intervening g. That is why we need to make <i>g</i> concrete, so that we can fold it out. We keep <i>f</i> polymorphic.
</div>

<h4>Maybe Transformer</h4>

In [7]:
newtype MaybeT m a = MaybeT { runMaybeT :: m (Maybe a) }

In [8]:
instance Functor m => Functor (MaybeT m) where
    fmap :: (a -> b) -> MaybeT m a -> MaybeT m b
    fmap f (MaybeT ma) = MaybeT $ (fmap.fmap) f ma

In [9]:
instance Applicative m => Applicative (MaybeT m) where
    pure :: a -> MaybeT m a
    pure a = MaybeT $ pure (Just a)
    
    (<*>) :: MaybeT m (a -> b) -> MaybeT m a -> MaybeT m b
    (<*>) (MaybeT fma) (MaybeT ma) = MaybeT $ fmap (<*>) fma <*> ma
    
{--
fma ~ m (Maybe (a -> b))
ma ~ m (Maybe a)

Applicative for m is of type
m (f a' -> f b') -> m (f a') -> m (f b')

fmap :: (a -> b) -> f2 a -> f2 b
(<*>) :: f1 (a -> b) -> f1 a -> f1 b
:t fmap (<*>) =>
(Applicative f1, Functor f2) => f2 (f1 (a -> b)) -> f2 (f1 a -> f1 b)

Therefore, fmap (<*>) fma :: m (Maybe a -> Maybe b)
--}

In [10]:
instance (Monad m) => Monad (MaybeT m) where
    return = pure
    
    (>>=) :: MaybeT m a -> (a -> MaybeT m b) -> MaybeT m b
    (>>=) (MaybeT ma) f = 
        MaybeT $ do
            v <- ma
            case v of
                Nothing -> return Nothing
                Just y -> runMaybeT (f y)
    
{--
What is m's bind operator going to look like?
(m (Maybe a)) >>= (Maybe a -> m (Maybe b)) ~ (m (Maybe b))

Maybe a's bind operator
Maybe a >>= (a -> Maybe b) ~ (Maybe b)

What is f's type?
a -> MaybeT m b ~ MaybeT $ m (Maybe b)

What is fmap f (Maybe a) ?
Maybe $ MaybeT m b
--}

<h4>Either Transformer</h4>

In [11]:
newtype EitherT e m a = EitherT { runEitherT :: m (Either e a) }

In [12]:
instance Functor m => Functor (EitherT e m) where
    fmap :: (a -> b) -> EitherT e m a -> EitherT e m b
    fmap f (EitherT ema) = EitherT $ fmap (fmap f) ema

In [13]:
instance Applicative m => Applicative (EitherT e m) where
    pure :: a -> EitherT e m a
    pure a = EitherT $ pure (pure a)
    
    (<*>) :: EitherT e m (a -> b) -> EitherT e m a -> EitherT e m b
    (<*>) (EitherT emf) (EitherT ema) = EitherT $ fmap (<*>) emf <*> ema
    
{--
fmap :: (a -> b) -> f a -> f b
(<*>) :: f1 (a -> b) -> f1 a -> f1 b

fmap (<*>) :: f (f1 (a -> b)) -> f (f1 a -> f1 b)

emf ~ m (Either e (a -> b))
fmap (<*>) emf :: m (Either e a -> Either e b)
emf' ~ fmap (<*>) emf

ema ~ m (Either e a)
emf' <*> ema :: m (Either e b)
--}

In [14]:
instance Monad m => Monad (EitherT e m) where
    return = pure
    
    (>>=) :: EitherT e m a -> (a -> EitherT e m b) -> EitherT e m b
    (>>=) (EitherT ema) f = EitherT $ do
        ea <- ema
        case ea of
            Left e -> return (Left e)
            Right a -> runEitherT $ f a

In [15]:
swapEither :: Either e a -> Either a e
swapEither ea = case ea of
    Left e -> Right e
    Right a -> Left a
    
swapEitherT :: (Functor m) => EitherT e m a -> EitherT a m e
swapEitherT (EitherT ema) = EitherT $ fmap swapEither ema

In [16]:
z = EitherT [Left "Rohit"]
runEitherT $ swapEitherT z

[Right "Rohit"]

In [17]:
eitherT :: Monad m => (a -> m c) -> (b -> m c) -> EitherT a m b -> m c
eitherT fa fb (EitherT amb) = amb >>= either fa fb

In [18]:
eitherT (:[]) (:[]) z

["Rohit"]

<h4>Reader Transformer</h4>

In [19]:
newtype ReaderT r m a = ReaderT { runReaderT :: r -> m a }

In [20]:
instance Functor m => Functor (ReaderT r m) where
    fmap :: (a -> b) -> ReaderT r m a -> ReaderT r m b
    fmap f (ReaderT rma) = ReaderT $ \r -> fmap f (rma r)

In [21]:
instance Applicative m => Applicative (ReaderT r m) where
    pure :: a -> ReaderT r m a
    pure a = ReaderT (pure (pure a))
    
    (<*>) :: ReaderT r m (a -> b) -> ReaderT r m a -> ReaderT r m b
    --               (r -> m (a -> b)) ->   (r -> m a) ->    (r -> m b)
    (<*>) (ReaderT fmab) (ReaderT rma) = ReaderT $ fmap (<*>) fmab <*> rma

In [22]:
instance Monad m => Monad (ReaderT r m) where
    return = pure

    (>>=) :: ReaderT r m a -> (a -> ReaderT r m b) -> ReaderT r m b
    -- (r -> m a) -> (a -> (r -> m b)) -> (r -> m b)
    (>>=) (ReaderT rma) f = ReaderT $ \r -> do
        a <- rma r
        runReaderT (f a) r

<h4>State Transformer</h4>

In [23]:
newtype StateT s m a = StateT { runStateT :: s -> m (a, s) }

In [24]:
instance (Functor m) => Functor (StateT s m) where
    fmap :: (a -> b) -> StateT s m a -> StateT s m b
    fmap f (StateT sma) = StateT $ \s -> fmap (\(x, y) -> (f x, y)) (sma s)

In [25]:
instance Monad m => Applicative (StateT s m) where
    pure a = StateT $ \s -> pure (a, s)
    
    (<*>) :: StateT s m (a -> b) -> StateT s m a -> StateT s m b
    (<*>) (StateT fmab) (StateT sma) = StateT $ \s -> do
        a <- fmap fst (sma s)
        f <- fmap fst (fmab s)
        return (f a, s)

In [26]:
instance Monad m => Monad (StateT s m) where
    return = pure
    
    (>>=) :: StateT s m a -> (a -> StateT s m b) -> StateT s m b
    (>>=) (StateT sma) f = StateT $ \s -> do
        a <- fmap fst (sma s)
        runStateT (f a) s

<h4>Embedded Example</h4>

In [27]:
embedded :: MaybeT (EitherT String (ReaderT () IO)) Int
embedded = return 1

In [28]:
maybeUnwrap :: EitherT String (ReaderT () IO) (Maybe Int)
maybeUnwrap = runMaybeT embedded

In [29]:
eitherUnwrap :: ReaderT () IO (Either String (Maybe Int))
eitherUnwrap = runEitherT maybeUnwrap

In [30]:
readerUnwrap :: () -> IO (Either String (Maybe Int))
readerUnwrap = runReaderT eitherUnwrap

In [31]:
readerUnwrap ()

Right (Just 1)

<h4>MonadTrans instances</h4>

In [32]:
class MonadTrans t where
    lift :: (Monad m) => m a -> t m a

In [33]:
instance MonadTrans IdentityT where
    lift = IdentityT

In [34]:
instance MonadTrans MaybeT where
    lift :: (Monad m) => m a -> MaybeT m a
    lift = MaybeT . fmap Just

In [35]:
instance MonadTrans (ReaderT r) where
    lift :: (Monad m) => m a -> ReaderT r m a
    lift = ReaderT . const

In [36]:
instance MonadTrans (EitherT e) where
    lift :: (Monad m) => m a -> EitherT e m a
    lift = EitherT . fmap Right

In [42]:
instance MonadTrans (StateT s) where
    lift :: (Monad m) => m a -> StateT s m a
    lift ma = StateT $ \s -> ma >>= (\x -> return (x, s))

<h4>More examples for showing the need for monad transformers</h4>
<h5><a href="https://en.wikibooks.org/wiki/Haskell/Monad_transformers">From this website</a></h5>

In [44]:
isValid :: String -> Bool
isValid s = length s >= 8
            && any isAlpha s
            && any isNumber s
            && any isPunctuation s

getPassphrase :: IO (Maybe String)
getPassphrase = do
    s <- getLine
    if isValid s then return $ Just s
                 else return Nothing

In [45]:
askPassphrase :: IO ()
askPassphrase = do
    putStrLn "Insert your new passphrase:"
    maybeVal <- getPassphrase
    case maybeVal of
        Just val -> putStrLn "Storing in db"
        Nothing -> putStrLn "Passphrase invalid"

<h6>Passphrase, simplified</h6>

In [64]:
getPassphraseUsingT :: MaybeT IO String
getPassphraseUsingT = do
    s <- lift getLine
    if isValid s then return s
                 else MaybeT $ return Nothing
                 
askPassphraseUsingT :: MaybeT IO String
askPassphraseUsingT = do
    lift $ putStrLn "Insert your new passphrase:"
    value <- getPassphraseUsingT
    lift $ putStrLn "Storing in db"
    return ""